In [1]:
import gymnasium as gym
from gymnasium.spaces import Discrete, Box, Sequence, Dict
import numpy as np
from src.commons import methods
import statistics

from src.rl_environments.env_impr import EnvImpr as Env

import os

## Config

In [2]:
TRAIN_CONFIG = {
    'alpha_range': [ (0 + i / 10) for i in range(1,10)],
    'beta_range': [round(i * 0.5, 1) for i in range(2, 6)],
    'h_range': [0.5],
    'c_range': [25],
    'total': range(10, 40), 
}

TEST_CONFIG = {
    'alpha_range': [ (3 + i / 10) for i in range(1,10)],
    'beta_range': [round(i * 0.5, 1) for i in range(2, 6)],
    'h_range': [0.5],
    'c_range': [25],
    'total': range(10, 40), 
}



In [3]:
PPO_MODEL_NAME = 'PPO_Env1'

## Train PQN

In [4]:
import stable_baselines3
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import SubprocVecEnv

if False:

    n_cpus = os.cpu_count()

    print(f"Number of processors: {n_cpus}")
    env = make_vec_env(lambda: Env(config=TRAIN_CONFIG), n_envs=n_cpus, vec_env_cls=SubprocVecEnv)
    model = PPO("MlpPolicy", env, verbose=1, device='cpu')

    print(model.n_envs)

    total_timesteps = 500_000
    print(f"Training model for {total_timesteps} timesteps")
    # Start training
    model.learn(total_timesteps=total_timesteps)

    model.save(methods.file_path(PPO_MODEL_NAME, 'models'))



## Generate Test Data

In [ ]:
def get_realized_data(config):
    alpha = np.random.choice(config['alpha_range'])
    beta = np.random.choice(config['beta_range'])
    h = np.random.choice(config['h_range'])
    c = np.random.choice(config['c_range'])
    total = np.random.choice(config['total'])
    intervals = np.random.gamma(shape=alpha, scale=beta, size=total)
    # travel_time = sum(intervals[4:]) - np.random.exponential(scale=alpha * beta)
    travel_time = sum(intervals[4:]) - np.random.gamma(shape=4, scale=alpha*beta)
    # travel_time = np.random.gamma(shape=5, scale=alpha*beta)
    # travel_time = max(beta * 5, travel_time)
    travel_time = sum(intervals[3:]) * np.random.uniform(0, 1)
    travel_time = max(alpha * beta, travel_time)

    return alpha, beta, h, c, total, intervals, travel_time

import pandas as pd

df = pd.DataFrame(columns=['h', 'c', 'travel_time', 'total', 'intervals', 'alpha', 'beta'])

for i in range(10000):
    alpha, beta, h, c, total, intervals, travel_time = get_realized_data(TEST_CONFIG)
    row = {'h': h, 'c': c, 'travel_time': travel_time, 'total': total, 'intervals': intervals, 'alpha': alpha, 'beta': beta}
    df.loc[i] = row



## Test

In [6]:
from stable_baselines3 import PPO
model = PPO.load(methods.file_path(PPO_MODEL_NAME, 'models'))

env = Env(config=TEST_CONFIG)
costs = {}
u_rl = {}
observed_n = {}
print(env._get_info())
for i in range(len(df)):
    row = df.iloc[i]
    state, _ = env.reset(row=row)
    done = False
    total_reward = 0
    while not done:
        action, _ = model.predict(state)  
        state, reward, done, _, info = env.step(action)
        total_reward += reward
    costs[i] = -total_reward
    u_rl[i] = env.cur_time + env.travel_time
    observed_n[i] = env.final_observed_n
    # print(f"Episode {i} reward: {total_reward} | h = {info['state']['h']}, c = {info['state']['c']}")
    print(f"Episode {i} reward: {total_reward} | cur_time = {info['state']['cur_time']}")

df['rl_cost'] = costs
df['u_rl'] = u_rl
df['rl_var_n_explore'] = observed_n
print(f"Average cost: {df['rl_cost'].mean()}")

{'hidden': {'alpha': -1, 'beta': -1, 'interval': -1, 'cum_sum_intervals': -1}, 'state': {'n': -1, 'N': -1, 'travel_time': -1, 'cur_time': -1, 'mean_n': -1, 'std_n': -1, 'u_star_hat': -1, 'last_update': -1, 'remain_ratio': -1, 'scaled_diff_u_tt': -1, 'scaled_diff_last_update': -1}}
Episode 0 reward: -7.977564668417969 | cur_time = 50.67201296305779
Episode 1 reward: -9.567993509879066 | cur_time = 10.188699098564891
Episode 2 reward: -3.3408258130149164 | cur_time = 62.0567423297693
Episode 3 reward: -5.501938300012746 | cur_time = 16.477489895980558
Episode 4 reward: -11.03662975348827 | cur_time = 18.703337279735074
Episode 5 reward: -11.72553201674691 | cur_time = 15.772628942579788
Episode 6 reward: -8.734477553613104 | cur_time = 28.81505414734552
Episode 7 reward: -25 | cur_time = 96.35313503466445
Episode 8 reward: -6.903723247162887 | cur_time = 17.1045803584482
Episode 9 reward: -5.0831490657640295 | cur_time = 12.76691475863187
Episode 10 reward: -13.426033567175779 | cur_time

## Add Comparison Metrics

In [7]:

from pandarallel import pandarallel

# Initialize pandarallel with progress bar enabled
pandarallel.initialize(progress_bar=True)


for i in [3, 5]:
    print(f"Oracle cost at fix n = {i}")
    df[f'oracle_cost_fix_n_{i}'] = df.parallel_apply(lambda row: methods.cal_oracle_cost_fix_n(alpha=row['alpha'],\
                                                                                     beta=row['beta'],\
                                                                                     intervals=row['intervals'],\
                                                                                     h=row['h'],\
                                                                                     c=row['c'],\
                                                                                     travel_time=row['travel_time'],\
                                                                                     n=i), axis=1)
    
    print(f"PTO cost at fix n = {i}")
    df[f'PTO_cost_fix_n_{i}'] = df.parallel_apply(lambda row: methods.cal_PTO_cost_fix_n(intervals=row['intervals'],\
                                                                                     h=row['h'],\
                                                                                     c=row['c'],\
                                                                                     travel_time=row['travel_time'],\
                                                                                     n=i), axis=1)
                                                                                
print(f"Oracle cost at variable n")
cost_and_explre = df.parallel_apply(lambda row: methods.cal_oracle_cost_var_n(alpha=row['alpha'],\
                                                                                     beta=row['beta'],\
                                                                                     intervals=row['intervals'],\
                                                                                     h=row['h'],\
                                                                                     c=row['c'],\
                                                                                     travel_time=row['travel_time']),result_type='expand', axis=1)

df['oracle_cost_var_n'], df['oracle_var_n_explore'] = cost_and_explre[0], cost_and_explre[1]

print(f"PTO cost at variable n")
cost_and_explre = df.parallel_apply(lambda row: methods.cal_PTO_cost_var_n(intervals=row['intervals'],\
                                                                                     h=row['h'],\
                                                                                     c=row['c'],\
                                                                                     travel_time=row['travel_time']),result_type='expand', axis=1)

df['PTO_cost_var_n'], df['PTO_var_n_explore'] = cost_and_explre[0], cost_and_explre[1]

df.head()

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.
Oracle cost at fix n = 3


PTO cost at fix n = 3


Oracle cost at fix n = 5


PTO cost at fix n = 5


Oracle cost at variable n


PTO cost at variable n


,h,c,travel_time,total,intervals,alpha,beta,rl_cost,u_rl,rl_var_n_explore,oracle_cost_fix_n_3,PTO_cost_fix_n_3,oracle_cost_fix_n_5,PTO_cost_fix_n_5,oracle_cost_var_n,oracle_var_n_explore,PTO_cost_var_n,PTO_var_n_explore
0,0.5,25,157.926845,34,"[4.572849187112826, 10.069113159171296, 4.4218...",3.7,2.0,7.977565,208.598857,8,25.000000,12.079273,25.000000,11.976772,25.000000,10.0,8.395092,8.0
1,0.5,25,105.416743,35,"[5.1909271614027315, 2.0353485584105697, 2.633...",3.8,1.0,9.567994,115.605442,3,8.438593,9.732327,5.745219,5.745219,8.438593,3.0,9.732327,3.0
2,0.5,25,67.493353,19,"[7.576754475133896, 2.9202045818973943, 7.7527...",3.8,2.0,3.340826,129.550095,8,6.027400,17.149780,4.596784,5.796056,4.105145,8.0,8.379207,6.0
3,0.5,25,43.830823,23,"[0.4022373707752396, 4.179314047805719, 5.2104...",3.4,1.0,5.501938,60.308313,6,2.622574,5.176573,3.598445,6.599494,4.433325,6.0,7.156323,4.0
4,0.5,25,221.474778,27,"[4.516057122594262, 3.546275248048314, 10.0376...",3.5,2.5,11.036630,240.178115,3,11.338296,11.338296,4.046537,4.046537,20.388298,0.0,11.338296,3.0


## Visualize Results

In [8]:
import plotly.graph_objects as go

all_costs = [x for x in df.columns if 'cost' in x] + ['oracle_var_n_explore', 'rl_var_n_explore', 'PTO_var_n_explore']

# Calculate means
means = df[all_costs + ['h', 'c']].mean()

# Plot
fig = go.Figure(data=[go.Bar(x=means.index, y=means.values)])
fig.update_layout(title="Mean Values of Costs, H, and C", yaxis_title="Mean Value")
fig.show()

# Calculate medians
medians = df[all_costs + ['h', 'c']].median()

# Plot
fig = go.Figure(data=[go.Bar(x=medians.index, y=medians.values)])
fig.update_layout(title="Median Values of Costs, H, and C", yaxis_title="Median Value")
fig.show()

# Create a dataframe with two rows: one for means and one for medians
summary_df = pd.DataFrame({'Mean': means, 'Median': medians}).T
summary_df.head()


,rl_cost,oracle_cost_fix_n_3,PTO_cost_fix_n_3,oracle_cost_fix_n_5,PTO_cost_fix_n_5,oracle_cost_var_n,PTO_cost_var_n,oracle_var_n_explore,rl_var_n_explore,PTO_var_n_explore,h,c
Mean,12.541720,10.401948,15.687165,8.987138,12.658172,10.516787,12.540323,5.1151,5.622,5.4002,0.5,25.0
Median,10.248288,7.974378,14.891298,5.796656,9.218127,8.481075,10.281669,5.0000,5.000,4.0000,0.5,25.0


In [9]:
df.head(50)
tp = len(df[df['oracle_cost_fix_n_5'] == 25])
print(tp)

1705
